# Showcase 管理（交互式表单）
使用 ipywidgets 在 Notebook 中以分离表单单元管理分类与事件：
- 分类：名称单元、图片单元、提交/更新单元
- 事件：标题单元、内容单元、图片单元、提交者单元、提交/更新单元

如未安装：在下方执行 `!pip install ipywidgets` 并重启内核。

In [1]:
# !pip install ipywidgets
from showcase_admin import ShowcaseAdmin
from ipywidgets import VBox, HBox, Text, Textarea, Dropdown, Button, Output, Layout, HTML
from IPython.display import display
import json

admin = ShowcaseAdmin()
out_cat = Output(layout=Layout(border='1px solid #eee'))
out_evt = Output(layout=Layout(border='1px solid #eee'))

## 分类管理表单（分开单元）

In [2]:
# 选择与刷新
cat_select = Dropdown(description='选择分类', options=[], layout=Layout(width='50%'))
btn_cat_refresh = Button(description='刷新分类列表', button_style='')

def refresh_categories(*_):
    cats = admin.list_categories()
    cat_select.options = [(f"{c.id} - {c.name}", c.id) for c in cats]
    if cats:
        cat_select.value = cats[0].id

btn_cat_refresh.on_click(refresh_categories)
refresh_categories()
display(HBox([cat_select, btn_cat_refresh]))
display(out_cat)

Output(layout=Layout(border_bottom='1px solid #eee', border_left='1px solid #eee', border_right='1px solid #ee…

In [4]:
# 名称单元
cat_name = Text(description='名称', placeholder='输入分类名称', layout=Layout(width='50%'))
display(cat_name)

Text(value='', description='名称', layout=Layout(width='50%'), placeholder='输入分类名称')

In [5]:
# 图片单元 - 支持粘贴
cat_img = Text(
    description='图片URL', 
    placeholder='https://... (支持 Ctrl+V 粘贴)', 
    layout=Layout(width='70%'),
    continuous_update=True,
    style={'description_width': 'initial'}
)
display(cat_img)

# 备用输入方法：如果粘贴不工作，可以使用这个
print("💡 如果无法粘贴，请手动输入URL或使用下方的替代方法")


Text(value='', description='图片URL', layout=Layout(width='70%'), placeholder='https://... (支持 Ctrl+V 粘贴)', styl…

💡 如果无法粘贴，请手动输入URL或使用下方的替代方法


In [6]:
# 备用输入方法：如果上面的文本框无法粘贴，使用这个方法
def set_cat_img_url():
    """手动设置分类图片URL的备用方法"""
    print("请在下面输入图片URL，然后按 Enter:")
    url = input("图片URL: ")
    if url.strip():
        cat_img.value = url.strip()
        print(f"✅ 已设置图片URL: {url.strip()}")
    else:
        print("❌ 未输入URL")


set_cat_img_url()


请在下面输入图片URL，然后按 Enter:
❌ 未输入URL


In [7]:
# 提交/更新单元
btn_create_cat = Button(description='创建分类', button_style='success')
btn_update_cat = Button(description='更新所选分类(留空字段不改)', button_style='warning')

def on_create_cat(_):
    with out_cat:
        out_cat.clear_output()
        if not cat_name.value.strip():
            print('请输入名称')
            return
        c = admin.create_category(cat_name.value.strip(), cat_img.value.strip() or None)
        print('创建成功:', c)
        refresh_categories()

def on_update_cat(_):
    with out_cat:
        out_cat.clear_output()
        if cat_select.value is None:
            print('请先选择分类')
            return
        new_name = cat_name.value.strip() or None
        new_img = cat_img.value.strip() or None
        try:
            c = admin.update_category(cat_select.value, name=new_name, image_url=new_img)
            print('更新成功:', c)
            refresh_categories()
        except Exception as e:
            print('更新失败:', e)

btn_create_cat.on_click(on_create_cat)
btn_update_cat.on_click(on_update_cat)
display(HBox([btn_create_cat, btn_update_cat]))

## 事件管理表单（分开单元）

In [8]:
# 选择分类与事件
ev_cat = Dropdown(description='分类', options=[], layout=Layout(width='50%'))
ev_select = Dropdown(description='事件', options=[], layout=Layout(width='70%'))
btn_ev_refresh = Button(description='刷新事件列表', button_style='')

def refresh_ev_categories(*_):
    cats = admin.list_categories()
    ev_cat.options = [(f"{c.id} - {c.name}", c.id) for c in cats]
    if cats:
        ev_cat.value = cats[0].id

def refresh_events(*_):
    if ev_cat.value is None:
        ev_select.options = []
        return
    evs = admin.list_events(ev_cat.value)
    ev_select.options = [(f"{e.id} - {e.title}", e.id) for e in evs]
    if evs:
        ev_select.value = evs[0].id

btn_ev_refresh.on_click(refresh_events)
refresh_ev_categories(); refresh_events()
display(HBox([ev_cat, btn_ev_refresh]))
display(ev_select)
display(out_evt)

Dropdown(description='事件', layout=Layout(width='70%'), options=(), value=None)

Output(layout=Layout(border_bottom='1px solid #eee', border_left='1px solid #eee', border_right='1px solid #ee…

In [9]:
# 标题单元
ev_title = Text(description='标题', placeholder='事件标题', layout=Layout(width='70%'))
display(ev_title)

Text(value='', description='标题', layout=Layout(width='70%'), placeholder='事件标题')

In [10]:
# 内容单元
ev_content = Textarea(description='内容', placeholder='正文，可多行', layout=Layout(width='90%', height='120px'))
display(ev_content)

Textarea(value='', description='内容', layout=Layout(height='120px', width='90%'), placeholder='正文，可多行')

In [11]:
# 图片单元（支持JSON数组或逗号分隔URL）- 支持粘贴
ev_images = Textarea(
    description='图片', 
    placeholder='["https://.../1.png", "https://.../2.png"] 或 逗号分隔 (支持 Ctrl+V 粘贴)', 
    layout=Layout(width='90%', height='80px'),
    continuous_update=True,
    style={'description_width': 'initial'}
)
display(ev_images)

# 备用输入提示
print("💡 如果无法粘贴，可以手动输入或使用下方的备用方法")

Textarea(value='', description='图片', layout=Layout(height='80px', width='90%'), placeholder='["https://.../1.p…

💡 如果无法粘贴，可以手动输入或使用下方的备用方法


In [12]:
# 备用输入方法：如果上面的文本框无法粘贴，使用这个方法
def set_event_images():
    """手动设置事件图片URL的备用方法"""
    print("请输入图片URL（支持多种格式）:")
    print("1. JSON数组格式: [\"https://...\", \"https://...\"]")
    print("2. 逗号分隔格式: https://..., https://...")
    print("3. 单个URL: https://...")
    urls = input("图片URL: ")
    if urls.strip():
        ev_images.value = urls.strip()
        print(f"✅ 已设置图片URL: {urls.strip()}")
    else:
        print("❌ 未输入URL")

# 如果粘贴不工作，运行这行代码：
# set_event_images()


In [13]:
# 提交者单元
ev_submitter = Text(description='提交者', placeholder='姓名/ID', layout=Layout(width='50%'))
display(ev_submitter)

Text(value='', description='提交者', layout=Layout(width='50%'), placeholder='姓名/ID')

In [ ]:
# 提交/更新单元
btn_create_ev = Button(description='创建事件', button_style='success')
btn_update_ev = Button(description='更新所选事件(留空字段不改)', button_style='warning')

def parse_images(val: str):
    v = (val or '').strip()
    if not v:
        return []
    try:
        if v.startswith('['):
            return json.loads(v)
    except Exception:
        pass
    # comma separated
    return [s.strip() for s in v.split(',') if s.strip()]

def on_create_ev(_):
    with out_evt:
        out_evt.clear_output()
        if ev_cat.value is None:
            print('请先选择分类')
            return
        if not ev_title.value.strip():
            print('请输入标题')
            return
        imgs = parse_images(ev_images.value)
        e = admin.create_event(ev_cat.value, ev_title.value.strip(), ev_content.value or None, imgs, ev_submitter.value or None)
        print('创建成功:', e)
        refresh_events()

def on_update_ev(_):
    with out_evt:
        out_evt.clear_output()
        if ev_select.value is None:
            print('请先选择事件')
            return
        kwargs = {}
        if ev_title.value.strip(): kwargs['title'] = ev_title.value.strip()
        if (ev_content.value or '').strip(): kwargs['content'] = ev_content.value
        if (ev_images.value or '').strip(): kwargs['images'] = parse_images(ev_images.value)
        if (ev_submitter.value or '').strip(): kwargs['submitted_by'] = ev_submitter.value.strip()
        try:
            e = admin.update_event(ev_select.value, **kwargs)
            print('更新成功:', e)
            refresh_events()
        except Exception as ex:
            print('更新失败:', ex)

btn_create_ev.on_click(on_create_ev)
btn_update_ev.on_click(on_update_ev)
display(HBox([btn_create_ev, btn_update_ev]))